## 지역명 코사인 유사도 

**목표**<br>
챗봇에서 지역명을 입력 받을 때, 데이터셋에 있는 지역명과 정확하게 일치하지 않아도 코사인 유사도 모델을 통해 비슷한 지역명을 선택할 수 있다.


- 가게리스트 데이터의 Region, Sigungu 컬럼을 이용해 지역명 데이터셋을 만든다.
- 위 데이터셋을 이용해 각 지역명별 코사인 유사도를 나타내는 모델을 만든다.


#### 코사인 유사도 설명 참고자료
- https://wikidocs.net/24603

In [1]:
import pandas as pd

import numpy as np
from numpy import dot
from numpy.linalg import norm

from konlpy.tag import Mecab
import re

### 1. 지역명 데이터 불러오기

In [2]:
# 가게리스트 데이터 불러오기
df_restaurants = pd.read_csv('./questions/가게리스트_카테고리_최종.csv',encoding='utf-8',index_col=0)
df_restaurants.head(2)

,storeid,Title,storename,Region,Sigungu,search_area,Category1,Category2,menu,Blog,reviews_num
0,13992,빛고을떡갈비,빛고을떡갈비,광주,광산구,광산,한식,숯불구이,육회비빔밥,24,2
1,13995,착한소장수,착한소장수,광주,동구,광주,한식,갈비,갈비살/안창살/살치살/왕갈비탕,4155,7


In [3]:
# Region 컬럼 저장
regions = df_restaurants.Region.unique().tolist()
regions

['광주',
 '전라북도',
 '전라남도',
 '경기도',
 '서울',
 '대전',
 '인천',
 '경상북도',
 '제주도',
 '부산',
 '강원도',
 '경상남도',
 '울산',
 '대구',
 '충청북도',
 '충청남도']

In [4]:
# Sigungu 컬럼 저장
sigungus = df_restaurants.Sigungu.unique().tolist()
# 중복 제거
sigungus = list(set(sigungus))
len(sigungus)

198

In [6]:
# Region + Sigungu 합쳐서 저장
reg_sig = df_restaurants.groupby(['Region','Sigungu'], as_index=False)\
          .count()[['Region','Sigungu']]
reg_sig['reg_sig'] = reg_sig.Region + ' ' + reg_sig.Sigungu
reg_sig.head(2)

,Region,Sigungu,reg_sig
0,강원도,강릉시,강원도 강릉시
1,강원도,고성군,강원도 고성군


In [7]:
# reg_sig 컬럼을 리스트로 변경
reg_sig_list = reg_sig['reg_sig'].tolist()
len(reg_sig_list)

229

In [8]:
# 지역명 리스트 통합
locations = regions + sigungus + reg_sig_list

# 중복 제거
locations = list(set(locations))
locations

['충청북도 옥천군',
 '고양시',
 '경산시',
 '경기도 파주시',
 '성북구',
 '강원도 삼척시',
 '서울 종로구',
 '영등포구',
 '서울 중구',
 '울산',
 '동대문구',
 '경기도',
 '경기도 성남시',
 '보성군',
 '충청남도 보령시',
 '부안군',
 '서울 서초구',
 '충청북도 증평군',
 '사하구',
 '경상남도 합천군',
 '인천 미추홀구',
 '경기도 이천시',
 '전라남도 장성군',
 '부평구',
 '달서구',
 '서울 마포구',
 '금산군',
 '해남군',
 '경상북도 상주시',
 '정선군',
 '여수시',
 '광산구',
 '진도군',
 '경상남도 남해군',
 '전라남도 담양군',
 '전라북도 순창군',
 '전라남도 무안군',
 '서울 용산구',
 '서울',
 '경기도 안성시',
 '충청남도',
 '남원시',
 '경상남도 사천시',
 '부산 해운대구',
 '대전 동구',
 '경기도 부천시',
 '대구 북구',
 '전주시',
 '부산 강서구',
 '송파구',
 '관악구',
 '울산 북구',
 '전라북도 종로구',
 '전라북도 남원시',
 '속초시',
 '인천 연수구',
 '부산 서구',
 '연수구',
 '경상북도 안동시',
 '전라북도 임실군',
 '평택시',
 '충청북도 진천군',
 '인천 계양구',
 '울산 동구',
 '부산 동구',
 '부산 남구',
 '옥천군',
 '김제시',
 '경기도 연천군',
 '경기도 의왕시',
 '경기도 의정부시',
 '김해시',
 '유성구',
 '완주군',
 '전라남도 목포시',
 '충청북도 음성군',
 '전라남도 화순군',
 '영주시',
 '경상남도 창원시',
 '강남구',
 '강원도 홍천군',
 '강원도 영월군',
 '중구',
 '부산 금정구',
 '서천군',
 '청송군',
 '경상남도 통영시',
 '경상북도 칠곡군',
 '경상남도 김해시',
 '부산 동래구',
 '경기도 광주시',
 '경상남도 함안군',
 '경상북도 예천군',
 '북구',
 '부여군',
 '서울 동대문구',
 '충청

In [9]:
# 가나다 순 정렬
locations = sorted(locations) ㅇ
locations

['가평군',
 '강남구',
 '강동구',
 '강릉시',
 '강북구',
 '강서구',
 '강원도',
 '강원도 강릉시',
 '강원도 고성군',
 '강원도 동해시',
 '강원도 삼척시',
 '강원도 속초시',
 '강원도 양구군',
 '강원도 양양군',
 '강원도 영월군',
 '강원도 원주시',
 '강원도 인제군',
 '강원도 정선군',
 '강원도 철원군',
 '강원도 춘천시',
 '강원도 태백시',
 '강원도 평창군',
 '강원도 홍천군',
 '강원도 화천군',
 '강원도 횡성군',
 '강진군',
 '강화군',
 '거제시',
 '거창군',
 '경기도',
 '경기도 가평군',
 '경기도 고양시',
 '경기도 과천시',
 '경기도 광명시',
 '경기도 광주시',
 '경기도 구리시',
 '경기도 군포시',
 '경기도 김포시',
 '경기도 남양주시',
 '경기도 동두천시',
 '경기도 부천시',
 '경기도 성남시',
 '경기도 수원시',
 '경기도 시흥시',
 '경기도 안산시',
 '경기도 안성시',
 '경기도 안양시',
 '경기도 양주시',
 '경기도 양평군',
 '경기도 여주시',
 '경기도 연천군',
 '경기도 오산시',
 '경기도 용인시',
 '경기도 의왕시',
 '경기도 의정부시',
 '경기도 이천시',
 '경기도 파주시',
 '경기도 평택시',
 '경기도 포천시',
 '경기도 하남시',
 '경기도 화성시',
 '경산시',
 '경상남도',
 '경상남도 거제시',
 '경상남도 거창군',
 '경상남도 고성군',
 '경상남도 김해시',
 '경상남도 남해군',
 '경상남도 밀양시',
 '경상남도 사천시',
 '경상남도 산청군',
 '경상남도 양산시',
 '경상남도 의령군',
 '경상남도 진주시',
 '경상남도 창녕군',
 '경상남도 창원시',
 '경상남도 통영시',
 '경상남도 하동군',
 '경상남도 함안군',
 '경상남도 함양군',
 '경상남도 합천군',
 '경상북도',
 '경상북도 경산시',
 '경상북도 경주시',
 '경상북도 구미시',
 '경상북도 군위군',
 

In [10]:
df_locations = pd.DataFrame({
  'location' : locations  
})
df_locations

,location
0,가평군
1,강남구
2,강동구
3,강릉시
4,강북구
...,...
438,홍천군
439,화성시
440,화순군
441,화천군


### 2. `TfidfVectorizer`를 이용하여 벡터화

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
QUESTION_RE = re.compile('[^ ㄱ-ㅣ가-힣a-zA-Z]+')

# 불용어는 챗봇 모델과 동일
STOPWORDS = set(['은','는','이','가','하','아','것','들','의','그','수','한','나','같','그렇'
                ,'문제','그리고','크','중','나오','지금','생각하','집','어떤','명','생각','이런'
                ,'인','지','을','를','에','스러운','스러워','주','할','만','게','도','져','된','로','고','던','로운','면서'
                ,'사실','이렇','점','싶','말','좀','식당','가게','집','음식점'
                ,'는지','나요','해요','해','는가요','삼','게요','예','는가','습니까','죠','려고요','는지요','서요','였어요','겠'
                ,'인가요','요' '라는','데','해서','세요','어요','을까요','건가요','겠죠','실래요','네요','으세요','지요','인데요'
                ,'드려요','려구요','합니다'])

# 각 지역명을 mecab 형태소 분석하기 위한 함수
def text_prepare(text):
    
    # Mecab 토크나이저
    mecab = Mecab()
    
    # mecab으로 text를 형태소 단위로 나누고 불용어를 제거
    text = ' '.join(token for token in mecab.morphs(text) if token not in STOPWORDS)

    return text

In [15]:
# 지역명에 mecab 형태소 분석 적용
df_locations['location_prep'] = df_locations.location.apply(text_prepare)
df_locations.head(10)

,location,location_prep
0,가평군,가평군
1,강남구,강남구
2,강동구,강동구
3,강릉시,강릉 시
4,강북구,강북구
5,강서구,강서구
6,강원도,강원도
7,강원도 강릉시,강원도 강릉 시
8,강원도 고성군,강원도 고성군
9,강원도 동해시,강원도 동해 시


In [16]:
# TF-IDF vectorizer 생성
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2) # unigram, bigram 
                                  ,token_pattern='(\S+)') # 공백을 제거한 모든 문자/숫자/특수문자

# 벡터화할 데이터를 저장
y = df_locations.location_prep.values

# 벡터화된 데이터의 지역명을 저장
y_location = df_locations.location.values

# vectorizer를 y 데이터로 fitting
tfidf_vectorizer.fit(y)

# fitting 된 vectorizer 확인
tfidf_vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(\\S+)', tokenizer=None,
                use_idf=True, vocabulary=None)

In [19]:
# tfidf에 사용된 형태소 단어 확인 : 443개 지역명을 나타내기 위해 총 497개가 사용되었음
tfidf_vocabs = tfidf_vectorizer.vocabulary_
len(tfidf_vocabs)

497

In [20]:
# 형태소 단어 미리보기
tfidf_vocabs

{'가평군': 0,
 '강남구': 1,
 '강동구': 2,
 '강릉': 3,
 '시': 280,
 '강릉 시': 4,
 '강북구': 5,
 '강서구': 6,
 '강원도': 7,
 '강원도 강릉': 8,
 '고성군': 109,
 '강원도 고성군': 9,
 '동해': 186,
 '강원도 동해': 10,
 '동해 시': 187,
 '삼척시': 234,
 '강원도 삼척시': 11,
 '속초시': 272,
 '강원도 속초시': 12,
 '양구군': 292,
 '강원도 양구군': 13,
 '양양군': 294,
 '강원도 양양군': 14,
 '영월군': 314,
 '강원도 영월군': 15,
 '원주시': 336,
 '강원도 원주시': 16,
 '인제군': 347,
 '강원도 인제군': 17,
 '정선군': 412,
 '강원도 정선군': 18,
 '철원군': 437,
 '강원도 철원군': 19,
 '춘천시': 442,
 '강원도 춘천시': 20,
 '태백시': 471,
 '강원도 태백시': 21,
 '평창군': 476,
 '강원도 평창군': 22,
 '홍천군': 492,
 '강원도 홍천군': 23,
 '화천군': 495,
 '강원도 화천군': 24,
 '횡성군': 496,
 '강원도 횡성군': 25,
 '강진군': 26,
 '강화군': 27,
 '거제시': 28,
 '거창군': 29,
 '경기도': 32,
 '경기도 가평군': 33,
 '고양시': 110,
 '경기도 고양시': 34,
 '과천시': 117,
 '경기도 과천시': 35,
 '광명시': 119,
 '경기도 광명시': 36,
 '광주시': 128,
 '경기도 광주시': 37,
 '경기': 30,
 '구리': 133,
 '경기 구리': 31,
 '구리 시': 134,
 '군포시': 138,
 '경기도 군포시': 38,
 '김포시': 144,
 '경기도 김포시': 39,
 '남양주시': 151,
 '경기도 남양주시': 40,
 '동두천시': 180,
 '경기도 동두천시': 41,
 '부천': 222,
 '경기도 부천

In [17]:
# y 데이터 벡터화
y_vectorized = tfidf_vectorizer.transform(y)

# 벡터화된 희소행렬 확인 
# (443*497 행렬, 443개 지역명이 총 497개의 형태소 벡터를 이용하여 벡터화되었음)
# (984개 element가 있으므로, 한 행당 평균 2개 정도의 element가 있음)
y_vectorized

<443x497 sparse matrix of type '<class 'numpy.float64'>'
	with 984 stored elements in Compressed Sparse Row format>

### 3. cosine 유사도 모델 생성 : 데이터 내부
- "y데이터 내부"에서 서로 비슷한 지역명은 무엇인지 찾는 모델 생성
- [linear_kernel](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.linear_kernel.html) 이용

In [21]:
from sklearn.metrics.pairwise import linear_kernel

# y_vectorized 데이터를 이용해 linear_kernel 행렬 생성
cosine_sim = linear_kernel(y_vectorized, y_vectorized)

In [23]:
cosine_sim.shape

(443, 443)

In [38]:
sim_scores = list(enumerate(cosine_sim[1])) 
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_list=[]
for i in range(len(sim_scores)):
    if sim_scores[i][1] > 0:
        sim_list.append(sim_scores[i])

In [39]:
sim_list

[(1, 1.0), (217, 0.6242026523122902)]

In [44]:
def get_similar_indices(idx, cosine_sim=cosine_sim):
    """
        idx를 "y데이터 내부"에서 서로 비슷한 지역명은 무엇인지 찾는 모델 생성
    """

    # 모든 지역명에 대해서 해당 idx와의 유사도를 구한다. ((index, linear_kernel값)의 list 형태)
    sim_scores = list(enumerate(cosine_sim[idx])) 

    # 유사도에 따라 지역명을 정렬한다. (key 파라미터는 정렬기준을 선택)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 유사도가 0 이상인 것만 sim_list에 저장한다.
    sim_list=[]
    for i in range(len(sim_scores)):
        if sim_scores[i][1] > 0:
            sim_list.append(sim_scores[i])
            
    # sim_list에서 유사도가 가장 높은 5개만 저장한다.
    sim_list = sim_list[:5]
            
    # sim_list의 인덱스를 저장한다.
    sim_indices = [i[0] for i in sim_list]

    # 5개의 지역명을 list 타입으로 반환한다.
    return sim_indices

In [46]:
test_idxs = [10,20,40,60,80,100,130,150,180,200]
for idx in test_idxs:
    test_loc = y_location[idx]
    sim_indices = get_similar_indices(idx)
    sim_loc = [y_location[idx] for idx in sim_indices]
    print("'",test_loc,"'",'가장 유사한 지역명 5개:', sim_loc)

' 강원도 삼척시 ' 가장 유사한 지역명 5개: ['강원도 삼척시', '삼척시', '강원도', '강원도 고성군', '강원도 양구군']
' 강원도 태백시 ' 가장 유사한 지역명 5개: ['강원도 태백시', '태백시', '강원도', '강원도 고성군', '강원도 삼척시']
' 경기도 부천시 ' 가장 유사한 지역명 5개: ['경기도 부천시', '부천시', '나주시', '경기도', '경기도 용인시']
' 경기도 화성시 ' 가장 유사한 지역명 5개: ['경기도 화성시', '화성시', '경기도', '경기도 가평군', '경기도 고양시']
' 경상남도 합천군 ' 가장 유사한 지역명 5개: ['경상남도 합천군', '합천군', '경상남도', '경상남도 고성군', '경상남도 거제시']
' 경상북도 포항시 ' 가장 유사한 지역명 5개: ['경상북도 포항시', '포항시', '경상북도', '나주시', '경상북도 경주시']
' 금산군 ' 가장 유사한 지역명 5개: ['금산군', '충청남도 금산군']
' 대구 남구 ' 가장 유사한 지역명 5개: ['대구 남구', '대구', '부산 남구', '인천 남구', '광주 남구']
' 보은군 ' 가장 유사한 지역명 5개: ['보은군', '충청북도 보은군']
' 부산진구 ' 가장 유사한 지역명 5개: ['부산진구', '부산 부산진구', '부산', '부산 중구', '부산 동구']


`locations_bio.csv` 파일 내부의 지역명에 대해서는 정확하게 유사한 지역명을 예측했다.

### 4. cosine 유사도 모델 생성 : 데이터 외부
- df_locations 데이터에 없던 지역명이 입력되었을 때, df_locations 내에서 가장 비슷한 지역명은 무엇인지 찾는 모델 생성
- [cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) 이용

#### 데이터에 없던 새로운 지역명을 테스트
- 현재 locations_bio.csv의 Region,Sigungu 값은 다음과 같다.
- Region 16개:
    - 강원도,경기도,경상남도,경상북도,광주,대구,대전,부산,서울,울산,인천,전라남도,전라북도,제주도,충청남도,충청북도
- Sigungu 198개:
    - 강릉시,고성군,동해시,등등등
- '서울특별시'/'울산광역시'/'강릉'/'동해'로 입력하면 각각 '서울'/'울산'/'강릉시'/'동해시'를 반환할 수 있는지 확인

In [47]:
from sklearn.metrics.pairwise import cosine_similarity

In [66]:
def get_similar_loc(new_loc, tfidf_vectorizer, y_vectorized):
    """
        데이터에 없는 새로운 지역명 new_loc을 입력 받고, 
        new_loc과 가장 유사한 지역명 3개를 리스트 형태로 반환
    """
    
    # new_loc 형태소 분석
    new_loc_mecab = [text_prepare(new_loc)]
    
    # new_loc 벡터화
    new_loc_vectorized = tfidf_vectorizer.transform(new_loc_mecab)
    
    # 
    sim_loc = {}

    for i in range(y_vectorized.shape[0]):
        curr_sim = cosine_similarity(y_vectorized[i], new_loc_vectorized)[0,0]
        if curr_sim > 0:
            sim_loc[y_location[i]] = curr_sim

    sim_loc = pd.DataFrame(data=sim_loc.values()
                          ,index=sim_loc.keys()
                          ,columns=['cs']).sort_values('cs',ascending=False)

    return sim_loc[:3]

In [67]:
test_locs = [
    '서울특별시','울산광역시','인천광역시','강릉','강릉시','동해','동해시','안양','안양시','종로','종로구에'
]

print('가장 유사한 지역명 3가지')
for loc in test_locs:
    print(loc,'\t:',get_similar_loc(loc,tfidf_vectorizer,y_vectorized).index.tolist())
    print('\tscore:',get_similar_loc(loc,tfidf_vectorizer,y_vectorized)['cs'].tolist())

가장 유사한 지역명 3가지
서울특별시 	: ['서울', '서울 중구', '서울 동작구']
	score: [1.0, 0.4398981974982893, 0.4213786091678332]
울산광역시 	: ['울산', '울산 중구', '울산 동구']
	score: [1.0, 0.5413737824674795, 0.5349423870789538]
인천광역시 	: ['인천', '인천 중구', '인천 동구']
	score: [1.0, 0.49289934044775574, 0.4866364067824756]
강릉 	: ['강릉시', '강원도 강릉시']
	score: [0.6419289807393734, 0.49789190221364515]
강릉시 	: ['강릉시', '강원도 강릉시', '나주시']
	score: [1.0000000000000002, 0.7756183583426528, 0.41934993427186606]
동해 	: ['동해시', '강원도 동해시']
	score: [0.6419289807393734, 0.4978919022136452]
동해시 	: ['동해시', '강원도 동해시', '나주시']
	score: [1.0000000000000002, 0.7756183583426529, 0.41934993427186606]
안양 	: []
	score: []
안양시 	: ['안양시', '경기도 안양시']
	score: [1.0, 0.6316762603982511]
종로 	: []
	score: []
종로구에 	: ['종로구', '서울 종로구', '전라북도 종로구']
	score: [1.0, 0.605406493522066, 0.6030739161299126]


mecab 형태소 분석시 '서울특별시' -> '서울''특별시'로 분리되기 때문에 tfidf_vocabs에 있던 '서울'이 감지되어 코사인유사도가 1.0이 나온다.

In [71]:
mecab.morphs('서울특별시')

['서울', '특별시']

In [74]:
get_similar_loc('서울특별시',tfidf_vectorizer,y_vectorized)

,cs
서울,1.000000
서울 중구,0.439898
서울 동작구,0.421379


'강릉시'도 '강릉''시'로 나눠지기 때문에 '강릉'을 검색하면 '강릉시'를 감지할 수 있다.

In [73]:
mecab.morphs('강릉시')

['강릉', '시']

In [75]:
get_similar_loc('강릉시',tfidf_vectorizer,y_vectorized)

,cs
강릉시,1.000000
강원도 강릉시,0.775618
나주시,0.419350


하지만 '안양시'는 '안양''시'로 나눠지기 않기 때문에 '안양'으로 검색했을 때 '안양시'를 감지할 수 없다.

In [72]:
mecab.morphs('안양시')

['안양시']

In [77]:
get_similar_loc('안양',tfidf_vectorizer,y_vectorized)

,cs


'종로' 지역명도 '종로구'를 감지할 수 없다.

In [88]:
get_similar_loc('종로',tfidf_vectorizer,y_vectorized)

,cs


**위 문제를 해결하기 위해 tfidf_vectorizer를 음절 단위로 나눠 적용해본다.**

In [81]:
# TF-IDF vectorizer 생성
tfidf_vectorizer2 = TfidfVectorizer(ngram_range=(2,8) # 2음절~8음절까지 검색
                                   ,analyzer='char')  # 음절 단위 분석

# 벡터화할 데이터(지역명)를 저장
y2 = df_locations.location.values

# vectorizer를 y2 데이터로 fitting
tfidf_vectorizer2.fit(y2)

# fitting 된 vectorizer 확인
tfidf_vectorizer2

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(2, 8), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [83]:
# tfidf에 사용된 음절 단어 확인 : 3,051개
tfidf_vocabs2 = tfidf_vectorizer2.vocabulary_
len(tfidf_vocabs2)

3051

In [86]:
# 음절 확인
tfidf_vocabs2

{'가평': 500,
 '평군': 3003,
 '가평군': 501,
 '강남': 502,
 '남구': 973,
 '강남구': 503,
 '강동': 504,
 '동구': 1597,
 '강동구': 505,
 '강릉': 506,
 '릉시': 1758,
 '강릉시': 507,
 '강북': 508,
 '북구': 1846,
 '강북구': 509,
 '강서': 510,
 '서구': 2163,
 '강서구': 511,
 '강원': 512,
 '원도': 2437,
 '강원도': 513,
 '도 ': 1197,
 ' 강': 3,
 '원도 ': 2438,
 '도 강': 1201,
 ' 강릉': 8,
 '강원도 ': 514,
 '원도 강': 2439,
 '도 강릉': 1202,
 ' 강릉시': 9,
 '강원도 강': 515,
 '원도 강릉': 2440,
 '도 강릉시': 1203,
 '강원도 강릉': 516,
 '원도 강릉시': 2441,
 '강원도 강릉시': 517,
 ' 고': 33,
 '고성': 790,
 '성군': 2242,
 '도 고': 1219,
 ' 고성': 34,
 '고성군': 791,
 '원도 고': 2442,
 '도 고성': 1220,
 ' 고성군': 35,
 '강원도 고': 518,
 '원도 고성': 2443,
 '도 고성군': 1221,
 '강원도 고성': 519,
 '원도 고성군': 2444,
 '강원도 고성군': 520,
 ' 동': 137,
 '동해': 1610,
 '해시': 3031,
 '도 동': 1296,
 ' 동해': 149,
 '동해시': 1611,
 '원도 동': 2445,
 '도 동해': 1302,
 ' 동해시': 150,
 '강원도 동': 521,
 '원도 동해': 2446,
 '도 동해시': 1303,
 '강원도 동해': 522,
 '원도 동해시': 2447,
 '강원도 동해시': 523,
 ' 삼': 206,
 '삼척': 2048,
 '척시': 2782,
 '도 삼': 1341,
 ' 삼척': 207,
 '삼척시': 2049,
 '원도 삼

In [84]:
# y2 데이터 벡터화
y2_vectorized = tfidf_vectorizer2.transform(y2)

# 벡터화된 희소행렬 확인 
# (443*3051 행렬, 443개 지역명이 총 3051개 벡터를 이용하여 벡터화되었음)
# (5703개 element가 있으므로, 한 행당 평균 13개 정도의 element가 있음)
y2_vectorized

<443x3051 sparse matrix of type '<class 'numpy.float64'>'
	with 5703 stored elements in Compressed Sparse Row format>

위에서 테스트한 지역명으로 재테스트

In [87]:
test_locs = [
    '서울특별시','울산광역시','인천광역시','강릉','강릉시','동해','동해시','안양','안양시','종로','종로구에'
]

print('가장 유사한 지역명 3가지')
for loc in test_locs:
    print(loc,'\t:',get_similar_loc(loc,tfidf_vectorizer2,y2_vectorized).index.tolist())
    print('\tscore:',get_similar_loc(loc,tfidf_vectorizer2,y2_vectorized)['cs'].tolist())

가장 유사한 지역명 3가지
서울특별시 	: ['서울', '서울 중구', '서울 동작구']
	score: [0.5731916684723996, 0.4168378565159031, 0.31938282491791387]
울산광역시 	: ['울산', '울산 동구', '울산 중구']
	score: [0.5189715213031163, 0.41874967946166985, 0.4144593137358266]
인천광역시 	: ['인천', '인천 남구', '인천 동구']
	score: [0.47448775390601877, 0.3863488800400197, 0.38596065911351785]
강릉 	: ['강릉시', '강원도 강릉시']
	score: [0.5773502691896257, 0.23203782544778032]
강릉시 	: ['강릉시', '경기도 시흥시', '강원도 강릉시']
	score: [0.3946879930184976, 0.18064050768582293, 0.1586256186541785]
동해 	: ['동해시', '강원도 동해시']
	score: [0.5937148690858357, 0.23505400459612888]
동해시 	: ['동해시', '경기도 시흥시', '강원도 동해시']
	score: [0.40587515518723033, 0.18064050768582293, 0.16068753800916014]
안양 	: ['안양시', '경기도 안양시']
	score: [0.6123131343857043, 0.24111915596886363]
안양시 	: ['안양시', '경기도 안양시', '고양시']
	score: [1.0, 0.39378406640054103, 0.25014525091750867]
종로 	: ['종로구', '서울 종로구', '전라북도 종로구']
	score: [0.5773502691896258, 0.25575289140593765, 0.19829489595853367]
종로구에 	: ['종로구', '서울 종로구', '전라북도 종로

음절 단위로 만든 tfidf_vectorizer는 '안양','종로'를 입력했을 때, '안양시','종로구'를 반환하므로 더 성능이 좋다고 할 수 있다.

**pickle 저장**

In [95]:
import pickle

with open('./cosine_loc_tfidf_vectorizer.pickle', "wb") as f:
    pickle.dump(tfidf_vectorizer2, f, pickle.HIGHEST_PROTOCOL)
with open('./cosine_loc_y_location.pickle', "wb") as f:
    pickle.dump(y2, f, pickle.HIGHEST_PROTOCOL)
with open('./cosine_loc_y_vectorized.pickle', "wb") as f:
    pickle.dump(y2_vectorized, f, pickle.HIGHEST_PROTOCOL)

**클래스 정리**